In [1]:
import pandas as pd
import numpy as np 
import altair as alt
from altair import datum
import geopandas as gpd
import matplotlib as mpl

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.lines as mlines

import random



import baltic as bt
from augur import frequencies,frequency_estimators


import datetime
from datetime import datetime as dt
from datetime import timedelta


import math
import time
import sys
import unicodedata
# import unidecode ## for removing diacritics from example geoJSON

import numpy as np
from scipy.interpolate import UnivariateSpline ## used to smooth counts of lineages in each location at any given time
from scipy.interpolate import interp1d ## used to linearly interpolate between data points used in colouring polygons
from sklearn.decomposition import IncrementalPCA ## used to identify PCA1 when automatically producing a colour map

import requests ## used to fetch examples from internet
import json ## used for importing JSONs
try:
    from StringIO import StringIO as sio
    from cStringIO import StringIO as csio
except ImportError:
    from io import StringIO as sio
    from io import BytesIO as csio
    


from Bio import Phylo

alt.data_transformers.disable_max_rows()


import json



In [ ]:
## unfortunately some of the mpox names are duplicated which makes baltic throw an error 
## use this code to depublicate and minimize it again!
def make_unique_name(name, existing_names):
    """Generate a unique name by appending a counter if a name already exists."""
    counter = 1
    new_name = name
    while new_name in existing_names:
        new_name = f"{name}_{counter}"
        counter += 1
    return new_name

def process_node(node, existing_names):
    """Recursively process nodes in the JSON to ensure unique names."""
    if isinstance(node, dict):
        if "name" in node:
            original_name = node["name"]
            if original_name in existing_names:
                new_name = make_unique_name(original_name, existing_names)
                print(f"Renaming {original_name} to {new_name}")
                node["name"] = new_name
            existing_names.add(node["name"])
        for key, value in node.items():
            if isinstance(value, (dict, list)):
                process_node(value, existing_names)
    elif isinstance(node, list):
        for item in node:
            process_node(item, existing_names)

def check_and_rename_nodes(input_file, output_file):
    """Process the JSON to deduplicate names and save in pretty format."""
    with open(input_file, 'r') as file:
        data = json.load(file)

    existing_names = set()
    process_node(data, existing_names)

    with open(output_file, 'w') as file:
        json.dump(data, file, indent=4)  # Save with pretty formatting

    print(f"Processing complete. Output written to {output_file}")

def minimize_json(input_file, minimized_output_file):
    """Minimize the JSON file by removing unnecessary formatting."""
    with open(input_file, 'r') as file:
        data = json.load(file)

    with open(minimized_output_file, 'w') as file:
        json.dump(data, file)  # Save without pretty formatting

    print(f"Minimized JSON written to {minimized_output_file}")


if __name__ == "__main__":
    input_file = '../mpox_build/phylogenetic/auspice/mpox_allcladeIIseqs.json'  # Replace with your actual input file path
    output_file = '../mpox_build/phylogenetic/auspice/mpox_allcladeIIseqs_deduplicated_tree.json'  # Replace with your desired output file path
    minimized_output_file = '../mpox_build/phylogenetic/auspice/mpox_allcladeIIseqs_deduplicated_tree_minimized.json'  # Output file for minimized JSON

    # Deduplicate names and save the JSON
    check_and_rename_nodes(input_file, output_file)

    # Minimize the JSON file
    minimize_json(output_file, minimized_output_file)
